In [1]:
# import libraries 
import pandas as pd
import csv
import numpy as np
from pathlib import Path
from basketball_reference_scraper.seasons import get_schedule
from itertools import product
from tkinter import *
import tkinter as tk
from tkinter import ttk

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
%matplotlib inline


## Data cleaning and preparation

In [4]:
# get 2019-2020, 2020-2021 nba playoffs schedule 
playoff2020 = get_schedule(2020, playoffs = True)
playoff2019 = get_schedule(2019, playoffs = True)
np.savetxt('2019playoff.csv', playoff2019, delimiter =",", fmt ="% s")
np.savetxt('2020playoff.csv', playoff2020, delimiter =",", fmt ="% s")

In [6]:
# load 2020 playoffs data
csv_path = Path('2020data.csv')
df_2020 = pd.read_csv(csv_path)
df_2020.head()

,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,(2019-20),8/17/2020 0:00,UTA,DEN,125,135,-10,40.1,85.1,0.47,...,10.08,32.56,42.64,25.37,7.19,4.44,14.26,20.47,109.18,160
1,(2019-20),8/17/2020 0:00,BRO,TOR,110,134,-24,40.4,90.3,0.45,...,10.92,35.14,46.06,25.57,7.22,5.49,16.83,20.36,106.51,450
2,(2019-20),8/17/2020 0:00,PHI,BOS,101,109,-8,41.1,87.9,0.47,...,10.32,34.01,44.33,22.40,7.14,5.50,15.24,20.65,107.35,230
3,(2019-20),8/17/2020 0:00,DAL,LAC,110,118,-8,41.7,90.3,0.46,...,10.71,34.60,45.31,23.74,7.50,4.69,14.07,22.88,109.90,285
4,(2019-20),8/18/2020 0:00,ORL,MIL,122,110,12,39.3,88.6,0.44,...,9.49,36.32,45.81,24.16,7.37,4.47,14.11,21.73,108.59,900


In [10]:
# create series matches unique id 

df_2020['matches'] = df_2020['VTeam']+df_2020['Hteam']
u_id = []
for i, r in df_2020.iterrows():
    if r.loc['matches'][0:3] < r.loc['matches'][3:7]:
        u_id_ = r.matches[0:3] + r.matches[3:7]
        u_id.append(u_id_)
    elif r.loc['matches'][0:3] > r.loc['matches'][3:7]:
        u_id_ = r.matches[3:7] + r.matches[0:3]
        u_id.append(u_id_)
#         print(False, r['u_id'])
df_2020['u_id'] = u_id
# create 'winner' column in the dataframe
def ss(df_2020):
    if df_2020['Mar.'] < 0 :
        return df_2020['Hteam']
    else:
        return df_2020['VTeam']
df_2020['winner'] = df_2020.apply(ss,axis=1)

df_2020.head()

,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line,matches,u_id,winner
0,(2019-20),8/17/2020 0:00,UTA,DEN,125,135,-10,40.1,85.1,0.47,...,25.37,7.19,4.44,14.26,20.47,109.18,160,UTADEN,DENUTA,DEN
1,(2019-20),8/17/2020 0:00,BRO,TOR,110,134,-24,40.4,90.3,0.45,...,25.57,7.22,5.49,16.83,20.36,106.51,450,BROTOR,BROTOR,TOR
2,(2019-20),8/17/2020 0:00,PHI,BOS,101,109,-8,41.1,87.9,0.47,...,22.40,7.14,5.50,15.24,20.65,107.35,230,PHIBOS,BOSPHI,BOS
3,(2019-20),8/17/2020 0:00,DAL,LAC,110,118,-8,41.7,90.3,0.46,...,23.74,7.50,4.69,14.07,22.88,109.90,285,DALLAC,DALLAC,LAC
4,(2019-20),8/18/2020 0:00,ORL,MIL,122,110,12,39.3,88.6,0.44,...,24.16,7.37,4.47,14.11,21.73,108.59,900,ORLMIL,MILORL,ORL


In [11]:
# sort dataframe by u_id and time
df_2020['Time'] = pd.to_datetime(df_2020['Time']).dt.date
df_2020 = df_2020.sort_values(["u_id", "Time"])
df_2020.reset_index()
df_2020['match'] = df_2020.u_id.eq(df_2020.u_id.shift())

In [13]:
# get series score
team_one = []
team_two = []

for i, r in df_2020.iterrows():
    if r['match'] == False:
        if r.loc['u_id'][0:3] == r.loc['winner']:
            team_one_ = 1
            team_two_ = 0
            team_one.append(team_one_)
            team_two.append(team_two_)
            
        elif r.loc['u_id'][3:7] == r.loc['winner']:
            team_one_ = 0
            team_two_ = 1
            team_one.append(team_one_)
            team_two.append(team_two_)
               
    elif r.loc['match'] == True:
        if r.loc['u_id'][0:3] == r.loc['winner']:
            team_one_ = team_one[-1] + 1
            team_two_ = team_two[-1]
            team_one.append(team_one_)
            team_two.append(team_two_)
        elif r.loc['u_id'][3:7] == r.loc['winner']:
            team_one_ = team_one[-1]
            team_two_ = team_two[-1] + 1
            team_one.append(team_one_)
            team_two.append(team_two_)
            
df_2020['team_one'] = team_one
df_2020['team_two'] = team_two

df_2020.head(5)

,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPTO,OPPF,OPPTS,H Betting Line,matches,u_id,winner,match,team_one,team_two
65,(2019-20),2020-09-15,MIA,BOS,117,114,3,39.5,84.4,0.47,...,15.24,20.65,107.35,115,MIABOS,BOSMIA,MIA,False,0,1
67,(2019-20),2020-09-17,MIA,BOS,106,101,5,39.5,84.4,0.47,...,15.24,20.65,107.35,135,MIABOS,BOSMIA,MIA,True,0,2
69,(2019-20),2020-09-19,BOS,MIA,117,106,11,41.3,89.6,0.46,...,14.00,21.70,109.10,-165,BOSMIA,BOSMIA,BOS,True,1,2
72,(2019-20),2020-09-23,BOS,MIA,109,112,-3,41.3,89.6,0.46,...,14.00,21.70,109.10,-165,BOSMIA,BOSMIA,MIA,True,1,3
74,(2019-20),2020-09-25,MIA,BOS,108,121,-13,39.5,84.4,0.47,...,15.24,20.65,107.35,140,MIABOS,BOSMIA,BOS,True,2,3


## Random Forest Training and Testing

In [22]:
# Set x variable list of features 
df_2020_x_var_list = ['FGM','FGA','FG%','3FGM','3FGA','3FG%','FTM','FTA','FT%','OFF',
                      'DEF','REB','AST','STL','BLK','TO','PF','PTS','OPFGM','OPFGA',
                      'OPFG%','OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%',
                      'OPOFF','OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF',
                      'OPPTS','H Betting Line']

In [23]:
# Construct the X_train and y_train datasets
df_2020_x_train = df_2020[df_2020_x_var_list]
df_2020_y_train = df_2020['Mar.']

In [24]:
# Set path to CSV and read 2021 playoffs data in CSV
csv_path = Path('2021playoff.csv')
df_2021 = pd.read_csv(csv_path)
df_2021.set_index('Time', inplace=True)
df_2021.head()

,Season,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,3FGM,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
Time,,,,,,,,,,,,,,,,,,,,,
5/22/2021 0:00,(2020-21),MIA,MIL,107.0,109.0,-2,39.22,83.74,0.47,12.94,...,8.04,33.46,41.50,26.32,7.90,3.97,14.07,18.93,108.07,190.0
5/22/2021 0:00,(2020-21),DAL,LAC,113.0,103.0,10,41.08,87.32,0.47,13.81,...,9.13,34.21,43.33,22.88,6.25,4.32,12.07,19.39,112.44,210.0
5/22/2021 0:00,(2020-21),BOS,BRO,93.0,104.0,-11,41.46,88.90,0.47,13.60,...,10.63,33.63,44.25,23.46,7.72,5.32,14.06,20.43,112.63,320.0
5/22/2021 0:00,(2020-21),POR,DEN,123.0,109.0,14,41.29,91.08,0.45,15.72,...,10.64,33.90,44.54,21.26,6.89,5.04,11.10,18.92,116.11,100.0
5/23/2021 0:00,(2020-21),WAS,PHI,118.0,125.0,-7,43.17,90.93,0.47,10.18,...,9.68,35.51,45.19,25.49,7.33,4.13,14.40,21.60,116.64,300.0


In [25]:
# Set x variable list of features
df_2021_x_var_list = ['FGM','FGA','FG%','3FGM','3FGA','3FG%','FTM','FTA','FT%','OFF',
                      'DEF','REB','AST','STL','BLK','TO','PF','PTS','OPFGM','OPFGA',
                      'OPFG%','OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%',
                      'OPOFF','OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF',
                      'OPPTS','H Betting Line']

In [26]:
# Construct testing start and end dates
playoff2021_testing_start =  '5/22/2021 0:00'
playoff2021_testing_end = '6/20/2021 0:00'

In [27]:
# Construct the X test and y test datasets
df_2021_test_X = df_2021[df_2021_x_var_list][playoff2021_testing_start:playoff2021_testing_end]
df_2021_test_X = df_2021_test_X.reset_index().drop(columns = 'Time')
df_2021_test_y = df_2021['Mar.'][playoff2021_testing_start:playoff2021_testing_end]
df_2021_test_X.head()

,FGM,FGA,FG%,3FGM,3FGA,3FG%,FTM,FTA,FT%,OFF,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,39.22,83.74,0.47,12.94,36.19,0.36,16.68,21.11,0.79,8.04,...,8.04,33.46,41.50,26.32,7.90,3.97,14.07,18.93,108.07,190.0
1,41.08,87.32,0.47,13.81,38.11,0.36,16.47,21.17,0.78,9.13,...,9.13,34.21,43.33,22.88,6.25,4.32,12.07,19.39,112.44,210.0
2,41.46,88.90,0.47,13.60,36.36,0.37,16.11,20.78,0.78,10.63,...,10.63,33.63,44.25,23.46,7.72,5.32,14.06,20.43,112.63,320.0
3,41.29,91.08,0.45,15.72,40.82,0.39,17.81,21.64,0.82,10.64,...,10.64,33.90,44.54,21.26,6.89,5.04,11.10,18.92,116.11,100.0
4,43.17,90.93,0.47,10.18,29.00,0.35,20.13,26.17,0.77,9.68,...,9.68,35.51,45.19,25.49,7.33,4.13,14.40,21.60,116.64,300.0


In [28]:
# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
df_2020_forest_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
df_2020_forest_model.fit(df_2020_x_train, df_2020_y_train)

RandomForestClassifier(max_depth=3, random_state=0)

In [29]:
# Make a prediction of "y" values from the X_test dataset
playoff2021_predictions = df_2020_forest_model.predict(df_2021_test_X)

In [30]:
# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
playoff2021_results = df_2021_test_y.to_frame()
playoff2021_results["Predicted Mar."] = playoff2021_predictions
playoff2021_results.head(5)

,Mar.,Predicted Mar.
Time,,
5/22/2021 0:00,-2,-10
5/22/2021 0:00,10,-2
5/22/2021 0:00,-11,-2
5/22/2021 0:00,14,8
5/23/2021 0:00,-7,-10


## Create GUI for our predictor

In [31]:
# load team stats data
teamstats2021_path = Path('2021teamseasonstats.csv')
df_team_2021 = pd.read_csv(teamstats2021_path)
df_team_2021 = df_team_2021.set_index('Team')
df_team_2021.head()

,League,Type,Season,Team/Opponent,Champion,Franchise,FGM,FGA,FG%,3FGM,...,OFF,DEF,REB,AST,STL,BLK,TO,PF,PTS,Unnamed: 25
Team,,,,,,,,,,,,,,,,,,,,,
ATL,NBA,Regular Season,(2020-21),Team Stats,NaN,Atlanta Hawks,40.8,87.2,0.47,12.4,...,10.6,35.1,45.6,24.1,7.0,4.8,13.2,19.3,113.7,NaN
BOS,NBA,Regular Season,(2020-21),Team Stats,NaN,Boston Celtics,41.5,88.9,0.47,13.6,...,10.6,33.6,44.3,23.5,7.7,5.3,14.1,20.4,112.6,NaN
BRO,NBA,Regular Season,(2020-21),Team Stats,NaN,Brooklyn Nets,43.1,87.3,0.49,14.2,...,8.9,35.5,44.4,26.8,6.7,5.3,13.5,19.0,118.6,NaN
CHA,NBA,Regular Season,(2020-21),Team Stats,NaN,Charlotte Hornets,39.9,87.8,0.45,13.7,...,10.6,33.2,43.8,26.8,7.8,4.8,14.8,18.0,109.5,NaN
CHI,NBA,Regular Season,(2020-21),Team Stats,NaN,Chicago Bulls,42.2,88.6,0.48,12.6,...,9.6,35.3,45.0,26.8,6.7,4.2,15.1,18.9,110.7,NaN


In [32]:
# Create UI for our prediction model
root = tk.Tk()
root.title('NBA Predictor')
root.geometry("555x350")

# Creat labels
# head labels
head = Label(root, text = "2020-21 NBA Playoffs Game Result Predictor", font = ('comic sans',20),
            fg = 'blue', relief = tk.RAISED, borderwidth = 3)
head.grid(row=0,column=0,columnspan=3)

row6 = Label(root, text = " ", font = ('comic sans',20))
row6.grid(row=6,column=0,columnspan=2) 

# team labels
team = Label(root, text = "Away Team     @     Home Team", font = ('comic sans',20))
team.grid(row=1,column=0,columnspan=2)             

# series score and betting line labels
ssbl = Label(root, text = "Series Score          Money Line", font = ('comic sans',20))
ssbl.grid(row=4,column=0,columnspan=2) 
seriesscore = ttk.Combobox(root,font = ('comic sans',20),width=8)
seriesscore['value'] = ('0-0','0-1','0-2','0-3','1-0','1-1','1-2','1-3',
                     '2-0','2-1','2-2','2-3','3-0','3-1','3-2','3-3')
seriesscore.grid(row=5,column=0,padx=5,pady=5,columnspan=1)

# user inputs
hometeam_var = ttk.Combobox(root,font = ('comic sans',20),width=8)
hometeam_var['value'] = ('ATL','BOS','BRO','DAL','DEN','LAC','LAL','MEM',
                     'MIA','MIL','NYK','PHI','PHO','POR','UTA','WAS')
hometeam_var.grid(row=2,column=0,padx=5,pady=5,columnspan=1)
awayteam_var = ttk.Combobox(root,font = ('comic sans',20),width=8)
awayteam_var['value'] = ('ATL','BOS','BRO','DAL','DEN','LAC','LAL','MEM',
                     'MIA','MIL','NYK','PHI','PHO','POR','UTA','WAS')
awayteam_var.grid(row=2,column=1,padx=5,pady=5,columnspan=1)

userin = IntVar()
bettingline= Entry(root,textvariable=userin,font = ('comic sans',20),width=9)
bettingline.grid(row=5,column=1,padx=5,pady=5,columnspan=1)

# def function to make prediction
def action():
    global DB
    import pandas as pd
    hometeam = hometeam_var.get()
    awayteam = awayteam_var.get()
    bettingline = userin.get()
    predictor_df = pd.DataFrame(columns=['FGM','FGA','FG%','3FGM',
                                         '3FGA','3FG%','FTM','FTA','FT%','OFF','DEF','REB','AST',
                                         'STL','BLK','TO','PF','PTS','OPFGM','OPFGA','OPFG%',
                                         'OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%','OPOFF',
                                         'OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF','OPPTS','H Betting Line'])

    
    FGM=df_team_2021.loc[hometeam,'FGM']
    predictor_df.loc[0,'FGM']=FGM
    
    FGA=df_team_2021.loc[hometeam,'FGA']
    predictor_df.loc[0,'FGA']=FGA
    
    FGP=df_team_2021.loc[hometeam,'FG%']
    predictor_df.loc[0,'FG%']=FGP
    
    TFGM=df_team_2021.loc[hometeam,'3FGM']
    predictor_df.loc[0,'3FGM']=TFGM
    
    TFGMA=df_team_2021.loc[hometeam,'3FGA']
    predictor_df.loc[0,'3FGA']=TFGMA
    
    TFGP=df_team_2021.loc[hometeam,'3FG%']
    predictor_df.loc[0,'3FG%']=TFGP
    
    FTM=df_team_2021.loc[hometeam,'FTM']
    predictor_df.loc[0,'FTM']=FTM
    
    FTA=df_team_2021.loc[hometeam,'FTA']
    predictor_df.loc[0,'FTA']=FTA
    
    FTP=df_team_2021.loc[hometeam,'FT%']
    predictor_df.loc[0,'FT%']=FTP
    
    OFF=df_team_2021.loc[hometeam,'OFF']
    predictor_df.loc[0,'OFF']=OFF
    
    DEF=df_team_2021.loc[hometeam,'DEF']
    predictor_df.loc[0,'DEF']=DEF
    
    REB=df_team_2021.loc[hometeam,'REB']
    predictor_df.loc[0,'REB']=REB
    
    AST=df_team_2021.loc[hometeam,'AST']
    predictor_df.loc[0,'AST']=AST
    
    STL=df_team_2021.loc[hometeam,'STL']
    predictor_df.loc[0,'STL']=STL
    
    BLK=df_team_2021.loc[hometeam,'BLK']
    predictor_df.loc[0,'BLK']=BLK
    
    TO=df_team_2021.loc[hometeam,'TO']
    predictor_df.loc[0,'TO']=TO
    
    PF=df_team_2021.loc[hometeam,'PF']
    predictor_df.loc[0,'PF']=PF
    
    PTS=df_team_2021.loc[hometeam,'PTS']
    predictor_df.loc[0,'PTS']=PTS
    
    OPFGM=df_team_2021.loc[awayteam,'FGM']
    predictor_df.loc[0,'OPFGM']=OPFGM
    
    OPFGA=df_team_2021.loc[awayteam,'FGA']
    predictor_df.loc[0,'OPFGA']=OPFGA
    
    OPFGP=df_team_2021.loc[awayteam,'FG%']
    predictor_df.loc[0,'OPFG%']=OPFGP
    
    OP3FGM=df_team_2021.loc[awayteam,'3FGM']
    predictor_df.loc[0,'OP3FGM']=OP3FGM
    
    OP3FGMA=df_team_2021.loc[awayteam,'3FGA']
    predictor_df.loc[0,'OP3FGA']=OP3FGMA
    
    OP3FGP=df_team_2021.loc[awayteam,'3FG%']
    predictor_df.loc[0,'OP3FG%']=OP3FGP
    
    OPFTM=df_team_2021.loc[awayteam,'FTM']
    predictor_df.loc[0,'OPFTM']=OPFTM
    
    OPFTA=df_team_2021.loc[awayteam,'FTA']
    predictor_df.loc[0,'OPFTA']=OPFTA
    
    OPFTP=df_team_2021.loc[awayteam,'FT%']
    predictor_df.loc[0,'OPFT%']=OPFTP
    
    OPOFF=df_team_2021.loc[awayteam,'OFF']
    predictor_df.loc[0,'OPOFF']=OPOFF
    
    OPDEF=df_team_2021.loc[awayteam,'DEF']
    predictor_df.loc[0,'OPDEF']=OPDEF
    
    OPREB=df_team_2021.loc[awayteam,'REB']
    predictor_df.loc[0,'OPREB']=OPREB

    OPAST=df_team_2021.loc[awayteam,'AST']
    predictor_df.loc[0,'OPAST']=OPAST

    OPSTL=df_team_2021.loc[awayteam,'STL']
    predictor_df.loc[0,'OPSTL']=OPSTL
    
    OPBLK=df_team_2021.loc[awayteam,'BLK']
    predictor_df.loc[0,'OPBLK']=OPBLK
    
    OPTO=df_team_2021.loc[awayteam,'TO']
    predictor_df.loc[0,'OPTO']=OPTO
    
    OPPF=df_team_2021.loc[awayteam,'PF']
    predictor_df.loc[0,'OPPF']=OPPF
    
    OPPTS=df_team_2021.loc[awayteam,'PTS']
    predictor_df.loc[0,'OPPTS']=OPPTS
    #bettingline = 150
    predictor_df.loc[0,'H Betting Line']=bettingline
    
    output=df_2020_forest_model.predict(predictor_df)
    if output >0:
        result = 'Away team will win'
    elif output <0:
        result = 'Home team will win'
    return result

# def function to display the prediction result
def result_display():
    result = action()
    result_display = tk.Text(master = root, height = 3, width=20)
    result_display.grid(row=7,column=1)
    result_display.insert(tk.END, result)

# make prediction button
predict = Button(root, text='Predict', font = ('comic sans',20), command=result_display)
predict.grid(row = 7, column =0)

root.mainloop()
